#Sociology of the Digital Public Space - Group Research - SCRAPING


Since Musk acquisition of Twitter/X at October 27th 2022 and especially since August 2023, the access of X's API has been completly blocked just as every simple solutions to scrap Twitter with well-known python library

In [ ]:
# author: Anthammax https://github.com/anthammax
# date: 28/11/2023
# last_update : 29/11/2023, 12:00:00

Github of the medialab library [HERE](https://github.com/medialab/minet/blob/master/docs/twitter.md)


# Imports

In [ ]:
%%capture
!pip install minet==1.0.0-a51
!pip install -U minet
#!pip install minet

In [ ]:
#!pip uninstall minet==1.0.0-a51
#!pip uninstall minet

In [ ]:
from minet.twitter import TwitterAPIScraper
from minet.twitter.constants import TWITTER_PUBLIC_API_AUTH_HEADER
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import json
from datetime import datetime, timedelta, date

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Functions

In [ ]:
# Functions ----> Tweet extraction

def test_scraper(cookies:dict):
  """ Test if the cookies are down or not"""

  for owner in list(cookies.keys()):
    scraper = TwitterAPIScraper(cookies[owner])
    headers = {
                "Authorization": TWITTER_PUBLIC_API_AUTH_HEADER,
                "Cookie": scraper.cookie,
                "Accept-Language": "en-US,en;q=0.5",
                "X-Csrf-Token": scraper.csrf_token,
                "X-Twitter-Active-User": "yes",
                "X-Twitter-Auth-Type": "OAuth2Session",
                "X-Twitter-Client-Language": "en",
            }
    url = "https://twitter.com/i/api/graphql/L1VfBERtzc3VkBBT0YAYHA/SearchTimeline?variables=%7B%22rawQuery%22%3A%22{}%22%2C%22count%22%3A20%2C%22querySource%22%3A%22typed_query%22%2C%22product%22%3A%22Latest%22%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Atrue%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Atrue%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3Atrue%2C%22view_counts_everywhere_api_enabled%22%3Atrue%2C%22longform_notetweets_consumption_enabled%22%3Atrue%2C%22responsive_web_twitter_article_tweet_consumption_enabled%22%3Afalse%2C%22tweet_awards_web_tipping_enabled%22%3Afalse%2C%22freedom_of_speech_not_reach_fetch_enabled%22%3Atrue%2C%22standardized_nudges_misinfo%22%3Atrue%2C%22tweet_with_visibility_results_prefer_gql_limited_actions_policy_enabled%22%3Atrue%2C%22longform_notetweets_rich_text_read_enabled%22%3Atrue%2C%22longform_notetweets_inline_media_enabled%22%3Atrue%2C%22responsive_web_media_download_video_enabled%22%3Afalse%2C%22responsive_web_enhance_cards_enabled%22%3Afalse%7D&fieldToggles=%7B%22withArticleRichContentState%22%3Afalse%7D"
    response = scraper.request(url, headers=headers)

    if response.status == 401:
      print(f"--/!\--The cookies from {owner} are down--/!\--")
    elif response.status == 200:
      print(f"The cookies from {owner} are available")
    else:
      print(f"--/!\--The cookies from {owner} have an identified issue--/!\--")


def test_build_scraper(cookies:dict):
  """ Test if the cookies are down or not and build the list of available scrapers"""
  scrapers = []
  for owner in list(cookies.keys()):
    scraper = TwitterAPIScraper(cookies[owner])
    headers = {
                "Authorization": TWITTER_PUBLIC_API_AUTH_HEADER,
                "Cookie": scraper.cookie,
                "Accept-Language": "en-US,en;q=0.5",
                "X-Csrf-Token": scraper.csrf_token,
                "X-Twitter-Active-User": "yes",
                "X-Twitter-Auth-Type": "OAuth2Session",
                "X-Twitter-Client-Language": "en",
            }
    url = "https://twitter.com/i/api/graphql/L1VfBERtzc3VkBBT0YAYHA/SearchTimeline?variables=%7B%22rawQuery%22%3A%22{}%22%2C%22count%22%3A20%2C%22querySource%22%3A%22typed_query%22%2C%22product%22%3A%22Latest%22%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Atrue%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Atrue%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3Atrue%2C%22view_counts_everywhere_api_enabled%22%3Atrue%2C%22longform_notetweets_consumption_enabled%22%3Atrue%2C%22responsive_web_twitter_article_tweet_consumption_enabled%22%3Afalse%2C%22tweet_awards_web_tipping_enabled%22%3Afalse%2C%22freedom_of_speech_not_reach_fetch_enabled%22%3Atrue%2C%22standardized_nudges_misinfo%22%3Atrue%2C%22tweet_with_visibility_results_prefer_gql_limited_actions_policy_enabled%22%3Atrue%2C%22longform_notetweets_rich_text_read_enabled%22%3Atrue%2C%22longform_notetweets_inline_media_enabled%22%3Atrue%2C%22responsive_web_media_download_video_enabled%22%3Afalse%2C%22responsive_web_enhance_cards_enabled%22%3Afalse%7D&fieldToggles=%7B%22withArticleRichContentState%22%3Afalse%7D"
    response = scraper.request(url, headers=headers)

    if response.status == 401:
      print(f"--/!\--The cookies from {owner} are down--/!\--")
    elif response.status == 200:
      print(f"The cookies from {owner} are available")
      scrapers.append(scraper)
    else:
      print(f"--/!\--The cookies from {owner} have an identified issue--/!\--")

  if len(scrapers) == 0:
    print()
    print("No possible scraper available !")
  else:
    print()
    print(f"--There are {len(scrapers)} cookies ready for scrap---")

  return scrapers

def final_scraper(scraper_list:list, limit:int, loop_limit:int):
  """Function returns a list of dictionaries containing in the scraper used and its own limits"""

  nb_need_scraper = (limit // loop_limit) if (limit % loop_limit) == 0 else 1 + (limit // loop_limit)
  trans_list = scraper_list * ((limit // (loop_limit*len(scraper_list))) + 1)

  final_scraper = []
  for i in range(nb_need_scraper):
    dict_scraper = {
        'scraper':  trans_list[i],
        'limit' : loop_limit if i != (nb_need_scraper - 1) else (limit- loop_limit*(nb_need_scraper-1))
    }
    final_scraper.append(dict_scraper)

  return final_scraper


def convert_date_time(starting_date:str):
  """Check if the date format is ok and convert the date into datetime format"""

  #Check if the date has the correct format
  try:
    date.fromisoformat(starting_date)
  except ValueError:
    raise ValueError("Incorrect data format, should be YYYY-MM-DD")

  return int(time.mktime(datetime.strptime(starting_date, "%Y-%m-%d").timetuple())) #by the default the time is YYYY-MM-DD-00:00


def big_query(hashtags_list:list, sensitive=False, since_time=False, until_time=False, froom=False, to=False, lang=False):
  """Function returning a query with several keywords or hashtags to have all the data in a single dataset"""

  query = ''
  for word in hashtags_list:
    query += ' ' + word
    query += ' OR' if word != hashtags_list[-1] else ''

  if sensitive == 'only':
     query += ' -filter:safe'
  elif sensitive == 'none':
    query += ' filter:safe'

  query += f' since_time:{since_time}' if since_time else ''
  query += f' until_time:{until_time}' if until_time else ''
  query += f' lang:{lang}' if lang else ''
  query += f' to:{to}' if to else ''
  query += f' from:{froom}' if froom else ''

  return query


def build_scraper(cookies:dict):
  """ Test if the cookies are down or not and build the list of available scrapers"""
  scrapers_dict = {}
  for owner in list(cookies.keys()):
    scraper = TwitterAPIScraper(cookies[owner])
    headers = {
                "Authorization": TWITTER_PUBLIC_API_AUTH_HEADER,
                "Cookie": scraper.cookie,
                "Accept-Language": "en-US,en;q=0.5",
                "X-Csrf-Token": scraper.csrf_token,
                "X-Twitter-Active-User": "yes",
                "X-Twitter-Auth-Type": "OAuth2Session",
                "X-Twitter-Client-Language": "en",
            }
    url = "https://twitter.com/i/api/graphql/L1VfBERtzc3VkBBT0YAYHA/SearchTimeline?variables=%7B%22rawQuery%22%3A%22{}%22%2C%22count%22%3A20%2C%22querySource%22%3A%22typed_query%22%2C%22product%22%3A%22Latest%22%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Atrue%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Atrue%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3Atrue%2C%22view_counts_everywhere_api_enabled%22%3Atrue%2C%22longform_notetweets_consumption_enabled%22%3Atrue%2C%22responsive_web_twitter_article_tweet_consumption_enabled%22%3Afalse%2C%22tweet_awards_web_tipping_enabled%22%3Afalse%2C%22freedom_of_speech_not_reach_fetch_enabled%22%3Atrue%2C%22standardized_nudges_misinfo%22%3Atrue%2C%22tweet_with_visibility_results_prefer_gql_limited_actions_policy_enabled%22%3Atrue%2C%22longform_notetweets_rich_text_read_enabled%22%3Atrue%2C%22longform_notetweets_inline_media_enabled%22%3Atrue%2C%22responsive_web_media_download_video_enabled%22%3Afalse%2C%22responsive_web_enhance_cards_enabled%22%3Afalse%7D&fieldToggles=%7B%22withArticleRichContentState%22%3Afalse%7D"
    response = scraper.request(url, headers=headers)

    if response.status == 200: #check if everythign's fine
      scrapers_dict[owner] = scraper

  if scrapers_dict == {}:
    return print("No cookies available")
  else:
    return scrapers_dict

def test_next_scraper(cookies:dict, current_owner:str):

  owners = list(cookies.keys())
  notok = True

  while notok:
    next_owner = owners[owners.index(current_owner) + 1] if owners.index(current_owner) != len(owners) - 1 else owners[0]

    scraper = TwitterAPIScraper(cookies[next_owner])
    headers = {
                "Authorization": TWITTER_PUBLIC_API_AUTH_HEADER,
                "Cookie": scraper.cookie,
                "Accept-Language": "en-US,en;q=0.5",
                "X-Csrf-Token": scraper.csrf_token,
                "X-Twitter-Active-User": "yes",
                "X-Twitter-Auth-Type": "OAuth2Session",
                "X-Twitter-Client-Language": "en",
            }
    url = "https://twitter.com/i/api/graphql/L1VfBERtzc3VkBBT0YAYHA/SearchTimeline?variables=%7B%22rawQuery%22%3A%22{}%22%2C%22count%22%3A20%2C%22querySource%22%3A%22typed_query%22%2C%22product%22%3A%22Latest%22%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Atrue%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Atrue%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3Atrue%2C%22view_counts_everywhere_api_enabled%22%3Atrue%2C%22longform_notetweets_consumption_enabled%22%3Atrue%2C%22responsive_web_twitter_article_tweet_consumption_enabled%22%3Afalse%2C%22tweet_awards_web_tipping_enabled%22%3Afalse%2C%22freedom_of_speech_not_reach_fetch_enabled%22%3Atrue%2C%22standardized_nudges_misinfo%22%3Atrue%2C%22tweet_with_visibility_results_prefer_gql_limited_actions_policy_enabled%22%3Atrue%2C%22longform_notetweets_rich_text_read_enabled%22%3Atrue%2C%22longform_notetweets_inline_media_enabled%22%3Atrue%2C%22responsive_web_media_download_video_enabled%22%3Afalse%2C%22responsive_web_enhance_cards_enabled%22%3Afalse%7D&fieldToggles=%7B%22withArticleRichContentState%22%3Afalse%7D"
    response = scraper.request(url, headers=headers)

    if response.status != 200:
      current_owner = next_owner
    else:

      notok = False

  return scraper, next_owner


def stopping_test(current_time:int, query:str):
  """Stopping test to know if the scrap should continue or not"""
  return current_time >=  int(query[query.find('since_time:')+11:query.find('since_time:')+21])

def time_before(shit_date:list, last_tweet_time:int, type_date:str, nb:int):
  """return th given time minus the time removed and update the list of dates skipped"""

  if type_date not in ['day','hour','minute','second']:
    raise Exception("Type of date not respected: must be 'day', 'hour, 'minute, or 'second'")

  if type_date == 'day':
    delta = timedelta(days=nb)
  elif type_date == 'hour':
    delta = timedelta(hours=nb)
  elif type_date == 'minute':
    delta = timedelta(minutes=nb)
  elif type_date == 'second':
    delta = timedelta(seconds=nb)

  final_time = datetime.fromtimestamp(last_tweet_time) - delta
  final_date = int(time.mktime((final_time.timetuple())))

  shit_date += [ i for i in range(final_date + 1, last_tweet_time + 1)]

  #For the moment, works only with seconds
  return shit_date, final_date

def modif_time(query: str, time:int):
  """Function that return a new query by chaging the end of the date of the query"""

  if query.find('until_time:') >= 0: #test if "until_until" is in the query
    old_end = query[query.find('until_time:')+11:query.find('until_time:')+21]
    new_query = query.replace(old_end, str(time))

  else:
    new_query = query + ' until_time:' + str(time)

  return new_query


def save_data_not_first_time(file_name:str, rows):
  """Function that adds the new tweets into the file called file_name, remove duplicates and give the real number of tweets added"""

  path = '//content/drive/Shareddrives/Group 3 - Sociology of Digital Public Spaces /Data/' + file_name + '.csv'

  prev_data = pd.read_csv(path)
  dataset = pd.concat([prev_data, pd.DataFrame.from_dict(rows, orient='columns')], axis=0)
  dataset = dataset.sort_values(by=['timestamp_utc']).reset_index(drop=True)
  dataset= dataset.drop_duplicates(subset=['url'])
  dataset.to_csv(path, index=False)

  return len(dataset) - len(prev_data)

# Functions ----> Dataviz



## Cookies
Process to get your personnal account cookies on [Twitter](https://pypi.org/project/ReverseTwitterScraper/)

In [ ]:
cookies = {
    #'anthony_ja' : 'twid=u%3D1708220042897211392; auth_token=e9249de2bb9c2de716c57ce652a9e662157312b4; ct0=6df589dec8d56fe91098598792d4ae19883ef7d4a8ef228a6946a44337b60a2f379e05b6ff770f791ce5e0848053a551fad60138d200e6240eb7c3641a86ec552ec2f7d8a24df7537cc0e8cabb054cd8; kdt=c01JksKwWBlREVlg6wIGYnBWs4gNh5x8gdAMWu6L; g_state={"i_l":0}; _ga=GA1.2.190970571.1698352412; _gid=GA1.2.1738961745.1698352412; guest_id=v1%3A169835267817359618; personalization_id="v1_7Jo+6f+WnqSD8JPHe0sBGg=="; lang=en; d_prefs=MToxLGNvbnNlbnRfdmVyc2lvbjoyLHRleHRfdmVyc2lvbjoxMDAw; guest_id_ads=v1%3A169797460158476146; guest_id_marketing=v1%3A169797460158476146'
    'anthony_perso' : 'twid=u%3D1579407475052249088; lang=en; _ga=GA1.2.1700078828.1699780642; _gid=GA1.2.2018183512.1699780642; auth_token=1b459a420f748ea2bfb11406a648da6ee3f824d1; ct0=ffd954ab9d04041f36404071cdc3ff5fba645d488f22f56d2a0dafd25bc3441d9b701c7ea44c659f450af24e731acfd459391db80bbf597b62842278a2c7b23cba86e83f150a2f985c0fc5f079faa1f2; kdt=gV9qtcUOcAXSXAKSyY8pHccAV4dPRbp9vNyQDU83; guest_id_ads=v1%3A169978063552215405; guest_id_marketing=v1%3A169978063552215405; personalization_id="v1_HQpRAAvYif1lEPPganbIYA=="; g_state={"i_l":0}; d_prefs=MToxLGNvbnNlbnRfdmVyc2lvbjoyLHRleHRfdmVyc2lvbjoxMDAw; guest_id=v1%3A169978063552215405'
    ,'christian_perso' : 'twid=u%3D1385120013489778689; auth_token=fb717edbbaeaef5388288422fb200e32421897b7; ct0=19f40a89dc128dc9bcb915067988e2c99a5c7ffed4751e92337aaa6dc68532a1b02fedd53c52f8c1b5b6845540ae25daaab25bb9948279ea3009f81a430970774c96bff4dedb1d8870dfc9f9245ecc3a; kdt=kr4AKfymftcKIcfid5jcboycAc3vMQ16YcBnVqDD; _twitter_sess=BAh7CSIKZmxhc2hJQzonQWN0aW9uQ29udHJvbGxlcjo6Rmxhc2g6OkZsYXNo%250ASGFzaHsABjoKQHVzZWR7ADoPY3JlYXRlZF9hdGwrCIOMG3GLAToMY3NyZl9p%250AZCIlZDE1Y2I4YjZiYzY1NDI0OWE4MDk3YzQ5NzMzMjEwM2Y6B2lkIiVmNmRi%250AN2Q4MjFiZjBjNGZjMzNiN2YwNGZmMDg5MGYxOA%253D%253D--dbb614d3d6505ca2ac61d5b392fb1cb30a66dd74; gt=1717880698437022185; guest_id=v1%3A169840964457353351; d_prefs=MjoxLGNvbnNlbnRfdmVyc2lvbjoyLHRleHRfdmVyc2lvbjoxMDAw; twtr_pixel_opt_in=N'
    ,'josh_perso' : 'guest_id=v1%3A168727034177752525; kdt=w27BVpZnDIeFfEbc9tPXu8fzWuDE0Dg9tp1yYkYO; auth_token=eb565e78507d8f28480a54bd08bc115fe96e010d; ct0=2dac1b5d6c1bd42c6d40c63cb600a4ab37b88bc46b75906e927cf0635760388d95d6f2095fffc00dfdfa42d3972107d9e77d49a5a447887a2f7b82cf29ea4034583af73a820b08ab6a8b25cd764cbc30; twid=u%3D1671161960828899329; d_prefs=MToxLGNvbnNlbnRfdmVyc2lvbjoyLHRleHRfdmVyc2lvbjoxMDAw; guest_id_ads=v1%3A168727034177752525; guest_id_marketing=v1%3A168727034177752525; personalization_id="v1_XxcCHomOEp1W2KBsGJdtdg=="; _ga=GA1.2.243836232.1695978210; external_referer=padhuUp37zjgzgv1mFWxJ12Ozwit7owX|0|8e8t2xd8A2w%3D;'
    ,'janine_perso' : 'twid=u%3D1214309638675423233; lang=en; auth_token=d645aa446fa81f5d641ea7986a3743710cfb2643; ct0=68eef42902cb83bb1caafdcdff9f349b53fe60fd4db3686d5089e41a06e85912e05c756ae42e4d01ceedd4c186518c9a1410b1befa5e41cdf396d2797582cbe20a15fccc6ea7526baaab73f0df7e4d3c; kdt=tfsy45KTt3pMFKAbd1c1u0G2CTNE9p7Ef18JEZHz; guest_id=v1%3A169627449563851578'
    ,'maria_perso' : 'guest_id=v1%3A169503219488028169; d_prefs=MjoxLGNvbnNlbnRfdmVyc2lvbjoyLHRleHRfdmVyc2lvbjoxMDAw; g_state={"i_p":1695039433702,"i_l":1}; kdt=3rF9GE6kKfySVtItPJD7cyih1sjgywyvA7QLP8gg; auth_token=fa9eeb51397427025f86157d2e4bfba2139d791a; ct0=9324e7b210055ed4f6d6cf953aac0671332bc278f0c07f5f0b32b2116cf2477e7c5e93bc21c62aebcc92e451ddfd588fb3f1d5a29bb21b7f0fe1c28e4e38e06b8bce9a0243193b1e291b4026b944001f; twid=u%3D1703717310958067712; lang=en'
    }

# Test  ad build of scrapers
scrapers = test_build_scraper(cookies)

# Massive Tweet Extraction

Unofficial documentation for queries [HERE](https://medium.com/@robguilarr/making-queries-to-twitter-api-on-tweepy-66afeb7184a4), [THERE](https://www.exportdata.io/blog/advanced-twitter-search-operators/)

More complexe query [HERE](https://twitter.com/search-advanced?f=live)

Documentation on the columns [HERE](https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet)

##Tests, find the relevant hashtags
In order to find relevant hashtags from a specific topic, the strategy is to start with several well-knwon sensitive hashtags, adress a query and study the other hashtags of the same tweets

In [ ]:
query_test ='#rotschildwishprivilege since_time:1635292800 until_time:1666656000'
scrapers_dict = build_scraper(cookies)
row_test = []
for tweet in scrapers_dict['janine_perso'].search_tweets(query=query):
      row_test.append(tweet)

df_test = pd.DataFrame(row_test)
df_test

In [ ]:
df_test = pd.DataFrame(row_test)
df_test

## General Extraction

METHOD
ref atisemistism = https://journals.sagepub.com/doi/10.1177/14614448221082122

### Continue the scraping

In [ ]:
# Scraping already begin
#file_name = 'try_19112013' from '2023-10-25' until 2023-10-10 17:49:32 en suspend car pas super utile
#file_name = 'before_attack_20112023' # done
file_name = 'before_Musk_23112023' # in process

json_path = '//content/drive/Shareddrives/Group 3 - Sociology of Digital Public Spaces /Data/' + file_name + '.json'
with open(json_path) as user_file:
 file_contents = user_file.read()
dict_info = json.loads(file_contents)

query = dict_info['query']
file_name = dict_info['file_name']
shit_date = dict_info['shit_date']

print(query)

 #rotschild OR #soros OR #zionist OR #thejew OR #jews OR #jew OR #jewishracialsupremacism OR #unbonjuif OR #yeisright OR #ye24 OR #jewishprivilege OR #killtheJews OR #hitlerwasright OR #bantheADL OR #exposethenose OR #jewishsupremacy OR #zionistjews OR #zionismnazism since_time:1635292800 until_time:1659956748


### New scraping

In [ ]:
# Parameters to implement
starting_date = '2022-10-28' #format AAAA-MM-DD required
ending_date = '2023-10-01' #format AAAA-MM-DD required
file_name = 'before_attack_20112023'

hashtag_list = [
    '#rotschild',
    '#soros',
    '#zionist',
    '#thejew',
    '#jews',
    '#jew',
    '#jewishracialsupremacism',
    '#unbonjuif',
    '#yeisright',
    '#ye24',
    '#jewishprivilege',
    '#killtheJews',
    '#hitlerwasright',
    '#bantheadl',
    '#exposethenose',
    '#jewishsupremacy',
    '#zionistjews',
    '#zionismnazism'
]

In [ ]:
# Columns
col = ['id','local_time','timestamp_utc','text','url','quoted_id','quoted_user','quoted_user_id','quoted_timestamp_utc','retweeted_id','retweeted_user','retweeted_user_id','retweeted_timestamp_utc','media_files','media_types','media_urls','media_alt_texts','links','links_to_resolve','domains','hashtags','mentioned_ids','mentioned_names','collection_time','match_query','collected_via','coordinates','to_tweetid','to_username','to_userid','lang','possibly_sensitive','retweet_count','like_count','reply_count','user_screen_name','user_name','user_friends','user_followers','user_location','user_verified','user_description','user_created_at','user_id','user_tweets','user_likes','user_lists','user_image','user_timestamp_utc','source_url','source_name','user_url']

# List of dates that hasn't been scraped because the cookies has crashed or the date (up to the second) is containing to many tweets to be scrpaed with a single cookies
# The amount of tweets will be estimated thanks to the number of tweets before ad after the problematic date
shit_date = []

# Building the query
starting_time, ending_time = convert_date_time(starting_date), convert_date_time(ending_date)

query = big_query(
    hashtags_list=hashtag_list,
    since_time=starting_time,
    until_time=ending_time
)

print("=> Your query has been built with success !")
print(f"Query: {query}")


dict_info = {
    'query': query,
    'file_name': file_name,
    'shit_date': shit_date
}

=> Your query has been built with success !
Query:  #rotschild OR #soros OR #zionist OR #thejew OR #jews OR #jew OR #jewishracialsupremacism OR #unbonjuif OR #yeisright OR #ye24 OR #jewishprivilege OR #killtheJews OR #hitlerwasright OR #bantheADL OR #exposethenose OR #jewishsupremacy OR #zionistjews OR #zionismnazism since_time:1635292800 until_time:1696118400


### Main work



In [ ]:
#Run as long as you're patient

# Initializing
rows = []
last_tweet_time = int(query[query.find('until_time:')+11:query.find('until_time:')+21])
scrapers_dict = build_scraper(cookies)
owner = list(scrapers_dict.keys())[0] # first available by default
scraper = scrapers_dict[owner]
first_time = True
t1 = 0
dict_test = {}


#Scraping
while stopping_test(last_tweet_time, query):

  try:
      if not first_time:
        print(f"=> Change of cookies:")
        print(f"---------> Execution time for {owner} cookies: {round(time.time() - t1, 2)}s")
        print(f"---------> Tweets scraped: {len(rows)}")
        print(f"---------> Current date: {datetime.fromtimestamp(last_tweet_time)}")
        scraper, owner = test_next_scraper(cookies, owner)
      else:
        first_time = False

      t1 = time.time()
      n_tweets = 0

      for tweet in scraper.search_tweets(query=query):
        rows.append(tweet)
        last_tweet_time = rows[-1]['timestamp_utc']
        n_tweets += 1

        if time.time() - t1 > 90: #time to sleep
          dict_test["a"]

  except KeyError:
    if n_tweets < 1: #nothing has changed, no more tweets
      print('/!\ -> Date changed artificially because of too many mistakes <- /!\'')
      last_tweet_time = time_before(last_tweet_time, seconds=1)
    query = modif_time(query, last_tweet_time)


In [ ]:
# Manually stop
if len(rows) > 0:
  print(f"Number of Tweets scraped1: {len(rows)}")
  print(f"The last date registered is: {datetime.fromtimestamp(rows[-1]['timestamp_utc'])}, id: {rows[-1]['timestamp_utc']}")
  diff =  save_data_not_first_time(file_name, rows)
  print("-----------------------")
  print("Data saved with sucmces")
  print(f" => {diff} Tweets added")
  print("-----------------------")
else:
  print(" => None Tweet has been registered")

choice = input("Do you wanna change the date? (Y/n): ")
if choice =='Y':
  choice_date = input("What type of date you wanna change ? (day,hour,minute,second): ")
  quantity = int(input("How many?: "))
  shit_date, last_tweet_time = time_before(shit_date, last_tweet_time, choice_date, quantity)
  query = modif_time(query, last_tweet_time)
  print(f"Your updated date is: {datetime.fromtimestamp(last_tweet_time)}, id: {last_tweet_time}")
else:
  query = modif_time(query, last_tweet_time)
  print(f"Your last date is: {datetime.fromtimestamp(last_tweet_time)}, id: {last_tweet_time}")
  print(f"Your query is: {query}")

#update dict
dict_info['query'] = query
dict_info['shit_date'] = shit_date
json_path = '//content/drive/Shareddrives/Group 3 - Sociology of Digital Public Spaces /Data/Scraping/' + file_name + '.json'
with open(json_path, "w") as outfile:
    json.dump(dict_info, outfile)

 => None Tweet has been registered
Do you wanna change the date? (Y/n): 
Your last date is: 2022-07-23 20:13:18, id: 1658607198
Your query is:  #rotschild OR #soros OR #zionist OR #thejew OR #jews OR #jew OR #jewishracialsupremacism OR #unbonjuif OR #yeisright OR #ye24 OR #jewishprivilege OR #killtheJews OR #hitlerwasright OR #bantheADL OR #exposethenose OR #jewishsupremacy OR #zionistjews OR #zionismnazism since_time:1635292800 until_time:1658607198


In [ ]:
# see the dataset
path = '//content/drive/Shareddrives/Group 3 - Sociology of Digital Public Spaces /Data/Scraping/' + file_name + '.csv'
dataset = pd.read_csv(path)
dataset

,id,local_time,timestamp_utc,text,url,quoted_id,quoted_user,quoted_user_id,quoted_timestamp_utc,retweeted_id,...,user_timestamp_utc,source_url,source_name,user_url,place_country_code,place_name,place_type,place_coordinates,lat,lng
0,1554612865373446144,2022-08-02T23:39:25,1659483565,Part 6/ Anti-Zionist Jews come out daily to #p...,https://twitter.com/NetureiKartaUS/status/1554...,NaN,NaN,NaN,NaN,NaN,...,1610316239,http://twitter.com/download/android,Twitter for Android,http://nkusa.org,NaN,NaN,NaN,NaN,NaN,NaN
1,1554615126459826177,2022-08-02T23:48:24,1659484104,#LGBTQ #propaganda #Soros #NYC #dragqueenstory...,https://twitter.com/SabrinaMedjane/status/1554...,1.554606e+18,stillgray,16106584.0,1.659482e+09,NaN,...,1633375842,http://twitter.com/download/android,Twitter for Android,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1554616258217349123,2022-08-02T23:52:53,1659484373,Paul’s brethren were Jews | #paul #letter #jew...,https://twitter.com/MikePetithomme/status/1554...,NaN,NaN,NaN,NaN,NaN,...,1350837454,http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1554617248085676033,2022-08-02T23:56:49,1659484609,@OnlinePalEng **\n\nJust look at way the fasci...,https://twitter.com/madayatollah/status/155461...,NaN,NaN,NaN,NaN,NaN,...,1389400477,https://mobile.twitter.com,Twitter Web App,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1554617962585374721,2022-08-02T23:59:40,1659484780,UN comédien peut jouer tous les rôles ! \nIl ...,https://twitter.com/RobertQc/status/1554617962...,NaN,NaN,NaN,NaN,NaN,...,1248486537,https://mobile.twitter.com,Twitter Web App,https://www.facebook.com/robert.bertrand.que,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20373,1585419342988316672,2022-10-26T23:53:21,1666828401,@wjlg4_wjlg Lol. I am literally laughing. Yo...,https://twitter.com/F45Danielle/status/1585419...,NaN,NaN,NaN,NaN,NaN,...,1436279304,http://twitter.com/download/iphone,Twitter for iPhone,https://www.amazon.com/hz/wishlist/ls/1KLI6Y73...,NaN,NaN,NaN,NaN,NaN,NaN
20374,1585419653593157633,2022-10-26T23:54:35,1666828475,@POTUS #Zionist puppet. You're just a bunch of...,https://twitter.com/RamondePronto/status/15854...,NaN,NaN,NaN,NaN,NaN,...,1663859095,https://mobile.twitter.com,Twitter Web App,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20375,1585420212987379716,2022-10-26T23:56:49,1666828609,The moment Kanye speaks out against #Soros and...,https://twitter.com/C1aytonG/status/1585420212...,NaN,NaN,NaN,NaN,NaN,...,1665640280,https://mobile.twitter.com,Twitter Web App,https://www.youtube.com/@CrimeInc,NaN,NaN,NaN,NaN,NaN,NaN
20376,1585420455338549248,2022-10-26T23:57:46,1666828666,Seems as if the very FIRST subject that Rapper...,https://twitter.com/inalienblwrites/status/158...,NaN,NaN,NaN,NaN,NaN,...,1652042044,https://mobile.twitter.com,Twitter Web App,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# first saving
path = '//content/drive/Shareddrives/Group 3 - Sociology of Digital Public Spaces /Data/Scraping/' + file_name + '.csv'

dataset = pd.DataFrame.from_dict(rows, orient='columns')
query = modif_time(query, last_tweet_time)
dataset= dataset.drop_duplicates(subset=['url'])
dataset.to_csv(path, index=False)
dataset